## Finance Condition Analyze

In [47]:
import pandas as pd

file_path = './data/finance.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

data = []
for line in lines:
    place, amount = line.strip().split(',')
    data.append([place, amount])

data

[['place', ' amount'],
 ['玉山', ' 3916'],
 ['郵局', ' 11706'],
 ['富邦', ' 2220'],
 ['國泰', ' 40027'],
 ['虛擬貨幣', ' 1332.03 (USD)'],
 ['台股 00878', ' 2500'],
 ['台股 台積', ' 165'],
 ['台股 玉山金', ' 4234'],
 ['台股 台達電', ' 85'],
 ['台股 緯創', ' 30'],
 ['台股 中租', ' 250'],
 ['台股 永昕', ' 500'],
 ['公司股', ' 200000']]

In [48]:
df = pd.DataFrame(data[1:], columns=["item", "amount"])
df

,item,amount
0,玉山,3916
1,郵局,11706
2,富邦,2220
3,國泰,40027
4,虛擬貨幣,1332.03 (USD)
5,台股 00878,2500
6,台股 台積,165
7,台股 玉山金,4234
8,台股 台達電,85
9,台股 緯創,30


In [53]:
import requests

def get_tse_stock_price(stock_symbol, market="tse"):
    url = f"https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch={market}_{stock_symbol}.tw"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        if "msgArray" in data and len(data["msgArray"]) > 0:
            price = data["msgArray"][0]["z"]  # 最新成交價
            return price
        else:
            return None
    except Exception as e:
        print(f"⚠️ 無法獲取 {market}_{stock_symbol} 的股價: {e}")
        return None

In [54]:
import requests

def get_usd_to_twd_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url, timeout=5) # set timeout to 5 seconds
        response.raise_for_status()  # check if the request is successful
        data = response.json()
        
        # check if the response data contains the exchange rates
        if "rates" in data and "TWD" in data["rates"]:
            return data["rates"]["TWD"]
        else:
            print("⚠️ API 回應異常，找不到 TWD 匯率！")
            return None
    except requests.exceptions.RequestException as e:
        print(f"⚠️ 無法獲取匯率: {e}")
        return None

usd_to_twd_rate = get_usd_to_twd_rate()

if usd_to_twd_rate is None:
    print("❌ 取得匯率失敗，使用預設匯率 31.5")
    usd_to_twd_rate = 31.5 # set default rate

print(f"💱 即時匯率: 1 USD = {usd_to_twd_rate:.2f} TWD")

💱 即時匯率: 1 USD = 32.88 TWD


In [55]:
stock_dict = {
    "00878": "00878",
    "台積": "2330",
    "玉山金": "2884",
    "台達電": "2308",
    "緯創": "3231",
    "中租": "5871",
    "永昕": "4726"
}

In [56]:
import re
# 初始化總台幣
total_twd = 0

# 處理每一筆數據
def convert_amount(item, amount_str):
    global total_twd

    # **確保 amount_str 是字串**
    if isinstance(amount_str, (int, float)):
        amount_str = str(amount_str)  # 轉為字串
    else:
        amount_str = amount_str.strip()  # 若已是字串，去除前後空格

    # **解析 純美金** (ex: "1332.03 (USD)")
    match_usd = re.match(r"([\d\.]+)\s*\(USD\)", amount_str)

    if match_usd:
        usd_value = float(match_usd.group(1))  # 取美金數字
        twd_amount = int(usd_value * usd_to_twd_rate)  # 美金轉台幣
        print(f"🔄 {usd_value} USD 轉換為 {twd_amount} TWD")
        total_twd += twd_amount
        return

    # **解析 台股 (ex: "台股 00878, 2500")**
    match_stock = re.match(r"台股\s+(.+)", item)
    
    if match_stock:
        stock_name = match_stock.group(1)  # 提取股票名稱
        try:
            shares = int(amount_str)  # 確保股數是 `int`
        except ValueError:
            print(f"⚠️ 無法解析股數: {amount_str}")
            return

        if stock_name in stock_dict:
            stock_symbol = stock_dict[stock_name]
            market = "tse" if stock_symbol != "4726" else "otc"
            
            # print(f"📈 查詢 {stock_name} ({stock_symbol}) 市場: {market}")
            stock_price = get_tse_stock_price(stock_symbol, market)

            if stock_price:
                try:
                    # **修正重複數字的問題**
                    stock_price = re.findall(r"[-+]?\d*\.\d+|\d+", stock_price)  # 取出數字
                    if stock_price:
                        stock_price = float(stock_price[0])  # 只取第一個正確數字
                    else:
                        raise ValueError("⚠️ 無法解析股價")

                    twd_value = int(stock_price * shares)  # 計算總價值
                    print(f"📈 {stock_name} ({stock_symbol}) {shares} 股，每股 {stock_price} TWD，總值: {twd_value} TWD")
                    total_twd += twd_value
                    return
                except ValueError:
                    print(f"⚠️ 股價格式錯誤: {stock_price}")
                    return

        print(f"⚠️ 無法獲取 {stock_name} ({stock_symbol}) 的股價")
        return

    # **純台幣**
    try:
        twd_amount = int(amount_str)
        print(f"💰 {item}: {twd_amount} TWD")
        total_twd += twd_amount
    except ValueError:
        print(f"⚠️ 無法解析數值: {amount_str}")

# **遍歷 DataFrame 並 `print` 出來**
for _, row in df.iterrows():
    convert_amount(row["item"], row["amount"])

# **最終總金額**
print(f"\n💰 總金額 (包含 USD & 台股換算): {total_twd} TWD")


💰 玉山: 3916 TWD
💰 郵局: 11706 TWD
💰 富邦: 2220 TWD
💰 國泰: 40027 TWD
🔄 1332.03 USD 轉換為 43801 TWD
📈 00878 (00878) 2500 股，每股 22.26 TWD，總值: 55650 TWD
📈 台積 (2330) 165 股，每股 1110.0 TWD，總值: 183150 TWD
📈 玉山金 (2884) 4234 股，每股 28.45 TWD，總值: 120457 TWD
📈 台達電 (2308) 85 股，每股 405.5 TWD，總值: 34467 TWD
📈 緯創 (3231) 30 股，每股 102.0 TWD，總值: 3060 TWD
📈 中租 (5871) 250 股，每股 115.5 TWD，總值: 28875 TWD
📈 永昕 (4726) 500 股，每股 40.55 TWD，總值: 20275 TWD
💰 公司股: 200000 TWD

💰 總金額 (包含 USD & 台股換算): 747604 TWD


### Modify Stock Number

In [63]:
# **篩選出所有 "台股" 開頭的項目**
stock_df = df[df["item"].str.startswith("台股")].reset_index()

# **列出可修改的台股**
print("\n📊 可修改的股票：")
for i, row in stock_df.iterrows():
    print(f"{i + 1}. {row['item']} - {row['amount']} 股")

# **使用者輸入選擇的股票**
try:
    choice = int(input("\n請輸入要修改的股票編號 (輸入 0 取消): "))
    if choice == 0:
        print("❌ 取消修改")
        exit()
    selected_stock = stock_df.loc[choice - 1, "item"]
except (ValueError, IndexError):
    print("⚠️ 輸入錯誤，請輸入正確的編號！")
    exit()

# **使用者輸入新的股數**
try:
    new_amount = int(input(f"🔄 請輸入新的 {selected_stock} 股數: "))
except ValueError:
    print("⚠️ 股數必須是數字！")
    exit()

# **更新 DataFrame**
df.loc[df["item"] == selected_stock, "amount"] = new_amount

# **顯示更新後的結果**
print("\n✅ 更新後的數據：")
# print(df.to_string(index=False))
print(df)


📊 可修改的股票：
1. 台股 00878 -  2500 股
2. 台股 台積 -  165 股
3. 台股 玉山金 -  4234 股
4. 台股 台達電 -  85 股
5. 台股 緯創 - 50 股
6. 台股 中租 -  250 股
7. 台股 永昕 -  500 股

✅ 更新後的數據：
        item          amount
0         玉山            3916
1         郵局           11706
2         富邦            2220
3         國泰           40027
4       虛擬貨幣   1332.03 (USD)
5   台股 00878            2500
6      台股 台積             165
7     台股 玉山金            4234
8     台股 台達電              85
9      台股 緯創              30
10     台股 中租             250
11     台股 永昕             500
12       公司股          200000


## Expenditure

In [46]:
import pandas as pd
import re

file_path = "./data/expenditure.txt"
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

data = []
for line in lines:
    match = re.match(r"(\d{4}/\d{1,2}/\d{1,2})\s+(.+?)\s+(\d+)", line.strip())
    if match:
        date, item, price = match.groups()
        data.append([date, item, int(price)])

df = pd.DataFrame(data, columns=["date", "item", "amount"])

In [47]:
# sum al of the price
total_price = df["amount"].sum()
print(f"total price: {total_price}")
df.loc[len(df)] = ["總計", "", total_price]

total price: 354515


In [48]:
# 儲存成 Excel
output_path = "./output/finance.xlsx"
df.to_excel(output_path, sheet_name='expenditure', index=False)

print(f"Excel saved at: {output_path}")

Excel saved at: finance.xlsx


In [49]:
# 儲存成 csv
output_path = "./output/expenditure.csv"
df.to_csv(output_path, index=False)

print(f"csv saved at: {output_path}")

csv saved at: ./output/expenditure.csv


In [50]:
df.tail(10)

,date,item,amount
133,2024/12/17,Ami 帽踢,7000
134,2024/12/17,Dior 大學踢,6000
135,2025/1/2,華泰名品城,5523
136,2025/1/4,Bumper (needles),13050
137,2025/1/10,花蓮,6875
138,2025/1/11,rooton 沐浴組,1900
139,2025/1/14,煙油,3100
140,2025/1/16,首飾,969
141,2025/1/20,balenciaga項鍊,3800
142,總計,,354515


## Income

In [64]:
new_file_path = "./data/income.txt"
with open(new_file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

data = []
for line in lines:
    match = re.match(r"([\w📈💰🏦🪙\s]+)\s+(\d+)", line.strip())
    if match:
        item, amount = match.groups()
        data.append(['??', item.strip(), int(amount)])

df_income = pd.DataFrame(data, columns=["date", "item", "amount"])
df_income.tail(10)

,date,item,amount
10,??,生日紅包,1000
11,??,地陪,2000
12,??,借學士服,500
13,??,網管紅包,600
14,??,生日紅包,20000
15,??,🏦股利,19012
16,??,📈股票,61523
17,??,🪙 虛擬貨幣,1277
18,??,💰台大薪水,120000
19,??,💰中研院薪水,94000


In [42]:
# sum al of the price
total_amount = df_income["amount"].sum()
print(f"total amout: {total_amount}")
df_income.loc[len(df_income)] = ["總計", "", total_amount]

total amout: 350812


In [35]:
# 讀取現有的 Excel 並新增 'income' Sheet
output_path = "./output/finance.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_income.to_excel(writer, sheet_name="income", index=False)

print(f"add 'income' Sheet to {output_path}")

add 'income' Sheet to finance.xlsx


In [44]:
# 儲存成 csv
output_path = "./output/income.csv"
df_income.to_csv(output_path, index=False)

print(f"csv saved at: {output_path}")

csv saved at: income.csv


## Add new data

In [15]:
import requests

def get_usd_to_twd_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url, timeout=5)  # 設置超時時間，避免無限等待
        response.raise_for_status()  # 檢查是否請求成功 (HTTP 200)
        data = response.json()
        
        # 檢查 API 回傳格式
        if "rates" in data and "TWD" in data["rates"]:
            return data["rates"]["TWD"]
        else:
            print("⚠️ API 回應異常，找不到 TWD 匯率！")
            return None
    except requests.exceptions.RequestException as e:
        print(f"⚠️ 無法獲取匯率: {e}")
        return None

# 讀取最新 USD to TWD 匯率
usd_to_twd_rate = get_usd_to_twd_rate()

# **修正：檢查 None，避免程式崩潰**
if usd_to_twd_rate is None:
    print("❌ 取得匯率失敗，使用預設匯率 31.5")
    usd_to_twd_rate = 31.5  # 你可以設定一個備用匯率，避免程式中斷

print(f"💱 即時匯率: 1 USD = {usd_to_twd_rate:.2f} TWD")

💱 即時匯率: 1 USD = 32.88 TWD


In [17]:
import pandas as pd

genre = input("請輸入類別 (1 for income / 2 for expenditure): ")
if genre == "1":
    csv_file = "./output/income.csv"
elif genre == "2":
    csv_file = "./output/expenditure.csv"

try:
    df = pd.read_csv(csv_file)
except FileNotFoundError:
    df = pd.DataFrame(columns=["date", "item", "amount"])

df.tail(10)

,date,item,amount
145,2025/01/14,煙油,3100
146,2025/01/16,首飾,969
147,2025/01/20,balenciaga項鍊,3800
148,2025/01/25,MOOS 衣服,3119
149,2025/01/25,Air Force,3240
150,2025/01/31,24/10/01 ~ 25/01/31 生活費,104926
151,2025/02/01,Bumper 皮衣 帽子,3960
152,2025/02/01,褲子 threads 店,790
153,2025/02/02,舊物盛典 手鍊 耳環,670
154,總計,NaN,678892


In [18]:
while True:
    raw_date = input("請輸入日期 (格式: YYYYMMDD，例如 20250120，輸入 q 退出): ")
    if raw_date.lower() == 'q':
        break

    try:
        formatted_date = f"{raw_date[:4]}/{int(raw_date[4:6])}/{int(raw_date[6:])}"
    except (ValueError, IndexError):
        print("⚠️ 格式錯誤，請輸入正確的 YYYYMMDD！")
        continue

    item = input("請輸入項目名稱: (if salary of genibuilder, please enter 1; if bonus, enter 2)")
    if item == "1":
        item = "Genibuilder 薪水"
    elif item == "2":
        item = "Genibuilder 獎金"

    amount = float(input("請輸入金額: "))

    try:
        amount = int(amount)
    except ValueError:
        print("⚠️ 金額必須是數字！")
        continue

    currency = input("enter u for USD")
    if currency == "u":
        amount = float(amount * usd_to_twd_rate)

    
    new_data = pd.DataFrame([[formatted_date, item, amount]], columns=["date", "item", "amount"])
    df = pd.concat([df[df["date"] != "總計"], new_data])

    df["date"] = pd.to_datetime(df["date"], errors='coerce')
    df = df.sort_values(by="date")

    df["date"] = df["date"].dt.strftime("%Y/%m/%d")

    total_amount = df["amount"].sum()
    df.loc[len(df)] = ["總計", "", total_amount]
    if genre == "1":
        print(f"總收入: {total_amount}", end='\n')
    elif genre == "2":
        print(f"總支出: {total_amount}", end='\n')

    df.to_csv(csv_file, index=False)

    print(f"✅ 已新增 {formatted_date} - {item} - {amount} 到 {csv_file}！")


總支出: 679742
✅ 已新增 2025/2/5 - 小莫 花🌹 - 850 到 ./output/expenditure.csv！


## Modify the data

In [1]:
import requests

def get_usd_to_twd_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url, timeout=5)  # 設置超時時間，避免無限等待
        response.raise_for_status()  # 檢查是否請求成功 (HTTP 200)
        data = response.json()
        
        # 檢查 API 回傳格式
        if "rates" in data and "TWD" in data["rates"]:
            return data["rates"]["TWD"]
        else:
            print("⚠️ API 回應異常，找不到 TWD 匯率！")
            return None
    except requests.exceptions.RequestException as e:
        print(f"⚠️ 無法獲取匯率: {e}")
        return None

# 讀取最新 USD to TWD 匯率
usd_to_twd_rate = get_usd_to_twd_rate()

# **修正：檢查 None，避免程式崩潰**
if usd_to_twd_rate is None:
    print("❌ 取得匯率失敗，使用預設匯率 31.5")
    usd_to_twd_rate = 31.5  # 你可以設定一個備用匯率，避免程式中斷

print(f"💱 即時匯率: 1 USD = {usd_to_twd_rate:.2f} TWD")

💱 即時匯率: 1 USD = 32.95 TWD


In [2]:
import pandas as pd

type = input("請輸入類別 (1 for income / 2 for expenditure): ")
if type == "1":
    csv_file = "./output/income.csv"
elif type == "2":
    csv_file = "./output/expenditure.csv"

try:
    df = pd.read_csv(csv_file)
except FileNotFoundError:
    print("⚠️ 找不到 finance.csv，請先新增一筆資料！")
    exit()

df.tail(10)

,date,item,amount
21,2024/01/01,💰 中研院薪水,94000
22,2024/08/10,Compal 薪水,32993
23,2024/09/10,Compal 薪水,32772
24,2024/10/09,Genibuilder 薪水,75000
25,2024/11/08,Genibuilder 薪水,105623
26,2024/12/10,Genibuilder 薪水,68123
27,2024/12/10,GenAI Stars 獎金,18000
28,2024/12/20,存款息,209
29,2025/01/10,Genibuilder 薪水,143123
30,總計,NaN,862872


In [3]:
print("\n📋 可選擇的項目:")
for i, row in enumerate(df.itertuples(), start=1):
    print(f"{i}. {row.date} - {row.item} - {row.amount}")


📋 可選擇的項目:
1. 2021/09/12 - 學測弟 家教 - 7700
2. 2022/06/27 - 小恩 家教 - 24100
3. 2022/10/26 - 宿舍杯 - 120
4. 2022/10/26 - 發票 - 3900
5. 2022/10/26 - 借學士服 - 500
6. 2022/10/26 - 地陪 - 2000
7. 2022/10/26 - 旋轉 - 12130
8. 2022/10/26 - 行政院 - 6000
9. 2022/10/26 - 生日禮物 - 3000
10. 2022/10/26 - 電資嘉年華 - 500
11. 2022/10/26 - 生日紅包 - 1000
12. 2022/10/26 - 葉配 - 1300
13. 2022/10/26 - Peeta硬舉 - 1000
14. 2022/10/26 - 網管紅包 - 600
15. 2022/10/26 - 出清台大 - 1950
16. 2022/10/26 - 程式接案 - 1000
17. 2023/10/26 - 生日紅包 - 20000
18. 2024/01/01 - 🪙 虛擬貨幣 - 3630
19. 2024/01/01 - 📈 股票 - 61523
20. 2024/01/01 - 🏦 股利 - 21076
21. 2024/01/01 - 💰 台大薪水 - 120000
22. 2024/01/01 - 💰 中研院薪水 - 94000
23. 2024/08/10 - Compal 薪水 - 32993
24. 2024/09/10 - Compal 薪水 - 32772
25. 2024/10/09 - Genibuilder 薪水 - 75000
26. 2024/11/08 - Genibuilder 薪水 - 105623
27. 2024/12/10 - Genibuilder 薪水 - 68123
28. 2024/12/10 - GenAI Stars 獎金 - 18000
29. 2024/12/20 - 存款息 - 209
30. 2025/01/10 - Genibuilder 薪水 - 143123
31. 總計 - nan - 862872


In [4]:
try:
    choice = int(input("\n請輸入要修改的項目編號 (輸入 0 退出): "))
    if choice == 0:
        print("❌ 取消修改")
        exit()
    selected_row = df.iloc[choice - 1]
except (ValueError, IndexError):
    print("⚠️ 輸入錯誤，請輸入正確的編號！")
    exit()

# 顯示選定的項目
print(f"\n🔹 你選擇了: {selected_row.date} - {selected_row.item} - {selected_row.amount}")


🔹 你選擇了: 2024/01/01 - <bound method IndexOpsMixin.item of date      2024/01/01
item          🪙 虛擬貨幣
amount          3630
Name: 17, dtype: object> - 3630


In [6]:
# 讓使用者選擇修改方式
while True:
    mode = input("請選擇更新方式: (1) 覆蓋金額 (2) 相加金額: ")
    if mode in ["1", "2", "q"]:
        break
    print("⚠️ 請輸入 1 或 2！")

# 讓使用者輸入新的金額
try:
    new_amount = float(input("請輸入新的金額: "))
except ValueError:
    print("⚠️ 金額必須是數字！")
    exit()

current = input("請輸入貨幣類型 (1 for TWD / 2 for USD): ")
if current == "2":
    new_amount = float(new_amount * usd_to_twd_rate)

# 根據選擇進行更新
if mode == "1":
    df.loc[df.index[choice - 1], "amount"] = new_amount
    print(f"✅ {selected_row.item} 的金額已被**覆蓋**為 {new_amount}！")
elif mode == "2":
    df.loc[df.index[choice - 1], "amount"] += new_amount
    print(f"✅ {selected_row.item} 的金額已**增加** {new_amount}，新總額為 {df.loc[df.index[choice - 1], 'amount']}！")
elif mode == "q":
    print("❌ 取消修改")
    exit()

# 重新計算總計
df = df[df["date"] != "總計"]
total_amount = df["amount"].sum()
df.loc[len(df)] = ["總計", "", total_amount]
print(f"✅ 總計已更新為 {total_amount}！")

# 存回 CSV
df.to_csv(csv_file, index=False)

print(f"📁 finance.csv 已更新！")

✅ <bound method IndexOpsMixin.item of date      2024/01/01
item          🪙 虛擬貨幣
amount          3630
Name: 17, dtype: object> 的金額已被**覆蓋**為 3630.0！
✅ 總計已更新為 862872.0！
📁 finance.csv 已更新！


## Count the Salary

In [24]:
import pandas as pd

csv_file = "./output/income.csv"
df = pd.read_csv(csv_file)

df = df[df["date"] != "總計"]

genibuilder_salary = df[df["item"] == "Genibuilder 薪水"]
total_salary = genibuilder_salary["amount"].sum()

print(f"💰 Genibuilder 薪水總金額: {total_salary}")

💰 Genibuilder 薪水總金額: 391869


## Analyze the amount after certain date

In [44]:
# csv_file = "./output/expenditure.csv"
csv_file = "./output/income.csv"
df = pd.read_csv(csv_file)

df = df[df["date"] != "總計"]

# 轉換日期格式，方便比較
df["date"] = pd.to_datetime(df["date"], format="%Y/%m/%d")

# 使用者輸入時間點
raw_date = input("請輸入日期 (格式: YYYYMMDD，例如 20250120，輸入 q 退出): ")

try:
    user_date = f"{raw_date[:4]}/{int(raw_date[4:6])}/{int(raw_date[6:])}"
except (ValueError, IndexError):
    print("⚠️ 格式錯誤，請輸入正確的 YYYYMMDD！")

# 過濾時間點之後的資料
df_after_date = df[df["date"] >= user_date]

# 計算總金額
total_amount_after = df_after_date["amount"].sum()

print(f"📅 {user_date} 之後的總金額: {total_amount_after}")
print()

if df_after_date.empty:
    print("⚠️ 沒有符合條件的項目！")
else:
    print(df_after_date.to_string(index=False))  # 顯示所有符合條件的資料
    print(f"\n💰 總金額: {total_amount_after}")

📅 2024/7/1 之後的總金額: 475843

      date           item  amount
2024-08-10      Compal 薪水   32993
2024-09-10      Compal 薪水   32772
2024-10-09 Genibuilder 薪水   75000
2024-11-08 Genibuilder 薪水  105623
2024-12-10 Genibuilder 薪水   68123
2024-12-10 GenAI Stars 獎金   18000
2024-12-20            存款息     209
2025-01-10 Genibuilder 薪水  143123

💰 總金額: 475843


In [45]:
csv_file = "./output/expenditure.csv"
# csv_file = "./output/income.csv"
df = pd.read_csv(csv_file)

df = df[df["date"] != "總計"]

# 轉換日期格式，方便比較
df["date"] = pd.to_datetime(df["date"], format="%Y/%m/%d")

# 使用者輸入時間點
raw_date = input("請輸入日期 (格式: YYYYMMDD，例如 20250120，輸入 q 退出): ")

try:
    user_date = f"{raw_date[:4]}/{int(raw_date[4:6])}/{int(raw_date[6:])}"
except (ValueError, IndexError):
    print("⚠️ 格式錯誤，請輸入正確的 YYYYMMDD！")

# 過濾時間點之後的資料
df_after_date = df[df["date"] >= user_date]

# 計算總金額
total_amount_after = df_after_date["amount"].sum()

print(f"📅 {user_date} 之後的總金額: {total_amount_after}")
print()

if df_after_date.empty:
    print("⚠️ 沒有符合條件的項目！")
else:
    print(df_after_date.to_string(index=False))  # 顯示所有符合條件的資料
    print(f"\n💰 總金額: {total_amount_after}")

📅 2024/7/1 之後的總金額: 375169

      date                    item  amount
2024-07-11                   寶寶 橘色    2079
2024-07-23                    紅軸鍵盤    2800
2024-07-25                 new era    1311
2024-07-25                    寶寶築間    1386
2024-07-25                   僔日本代購    4500
2024-07-27                      刺青    3000
2024-08-02                   香港 🇭🇰   25000
2024-08-06                Marshall    5475
2024-08-08                      罰單    1200
2024-08-23                   人體工學椅    7000
2024-08-24           balenciaga 墨鏡    7443
2024-08-25                     自拍棒     990
2024-08-28                   377美白    1200
2024-08-31                    螢幕支架    1300
2024-09-03                dyptique    4500
2024-09-04                      凱薩    1550
2024-09-11                   請客 燒肉    2893
2024-09-15                      隊費    2000
2024-09-22                     找女人    8157
2024-09-24                   煙油 煙倉    3000
2024-09-30                   找女人上來    2500
2024-10-01                 

In [4]:
import pandas as pd

file_path = "finance.xlsx"
sheets_dict = pd.read_excel(file_path, sheet_name=None)

for sheet_name, df in sheets_dict.items():
    print(f"Sheet: {sheet_name}")
    print(df.head())


Sheet: income
Empty DataFrame
Columns: []
Index: []
Sheet: expenditure
        date         item  price
0  2022/6/26           帽子    591
1  2022/7/18        小包 褲子   1500
2  2022/8/24           修鞋   1000
3  2022/9/28         ck禮物   1730
4  2022/10/2  Apple Watch   7000


In [5]:
df = pd.read_excel(file_path, sheet_name="expenditure")
df.tail(10)

,date,item,price
118,2024/12/16,擴香,650
119,2024/12/17,Ami 帽踢,7000
120,2024/12/17,Dior 大學踢,6000
121,2025/1/2,華泰名品城,5523
122,2025/1/4,Bumper (needles),13050
123,2025/1/10,花蓮,6875
124,2025/1/11,rooton 沐浴組,1900
125,2025/1/14,煙油,3100
126,2025/1/16,首飾,969
127,2025/1/20,balenciaga項鍊,3800
